In [1]:
# -*- coding: utf-8 -*-
import sys
from xml.dom import NamespaceErr
import cv2 as cv
import time
import winsound
import win32ui, win32con, win32api
import os.path
import xml.etree.ElementTree as et
import numpy as np


# DEBUG = False
DEBUG = True
DEBUG_INTERACTIVE = False

OUTPUT_DIR_NAME = "shot_snapshots"
soundfile = "ton.wav"

In [2]:

BLACK_AND_WHITE = False
THRESHOLD = 0.48
BW_THRESHOLD = 0.4

os.chdir("projects\AFRAID Short Film (AWARD-WINNING)")
try:
	os.mkdir(OUTPUT_DIR_NAME)
except:
	pass

if len(sys.argv) > 2:
	if sys.argv[2] == "bw":
		BLACK_AND_WHITE = True
		THRESHOLD = BW_THRESHOLD
		print("##########")
		print(" B/W MODE")
		print("##########")

tree = et.parse("project.xml")
movie = tree.getroot()
file_path = movie.attrib["path"]
cap = cv.VideoCapture(file_path)
if DEBUG:
	cv.namedWindow("win", cv.WINDOW_AUTOSIZE)
	cv.moveWindow("win", 200, 200)
hist = None
prev_hist = None
prev_img = None
pixel_count = None
frame_counter = 0
last_frame_black = False
black_frame_start = -1

t = time.time()


In [16]:

for i in range(150):
	retval, img_orig = cap.read()

if not retval: # eof
	cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (frame_counter-1), prev_img)
	"""movie.set("frames", str(frame_counter))
	tree.write("project.xml")"""

In [17]:

# img = cv.CreateImage((int(img_orig.width/4), int(img_orig.height/4)), cv.IPL_DEPTH_8U, 3)
img = np.zeros((int(img_orig.shape[1]/4), int(img_orig.shape[0]/4), 3), np.uint8)
img = cv.resize(img_orig, img.shape[:2], interpolation=cv.INTER_AREA)

if frame_counter == 0: # erster frame
	cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (0), img)
	pixel_count = img.shape[0] * img.shape[1]
	prev_img = np.zeros(img.shape, np.uint8)

if DEBUG and frame_counter % 2 == 1:
	cv.imshow("win", img)

# img_hsv = np.zeros(img.shape, np.uint8)
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

with np.printoptions(threshold=np.inf):
    print(img_hsv)

[[[ 30  49  21]
  [ 30  70  22]
  [ 24  58  22]
  [ 30  53  24]
  [ 42  51  25]
  [ 54  51  25]
  [ 40  64  24]
  [ 30  55  23]
  [ 39  81  22]
  [ 35  70  22]
  [ 30  58  22]
  [ 30  58  22]
  [ 30  58  22]
  [ 30  51  25]
  [ 30  41  31]
  [ 30  70  22]
  [ 30  54  19]
  [ 24  67  19]
  [ 26  99  18]
  [ 30 120  17]
  [ 23  60  17]
  [ 30  41  25]
  [ 34  78  23]
  [ 30  77  20]
  [ 30  58  22]
  [ 30  44  23]
  [ 30  55  23]
  [ 30  46  22]
  [ 30  46  22]
  [ 30  33  23]
  [ 30  53  24]
  [ 30  51  25]
  [ 30  28  27]
  [ 30  31  25]
  [ 30  31  25]
  [ 30  29  26]
  [ 30  27  28]
  [ 30  26  30]
  [ 30   8  32]
  [ 30  26  29]
  [ 30  36  28]
  [ 30  47  27]
  [ 30  38  27]
  [ 30  26  29]
  [ 30  26  29]
  [ 50  25  31]
  [ 50  23  33]
  [ 30   9  29]
  [ 30  27  28]
  [ 30  27  28]
  [ 30  28  27]
  [ 30  49  26]
  [ 30  32  24]
  [ 30  21  24]
  [ 30  27  28]
  [  0   0  30]
  [ 30  28  27]
  [ 30  47  27]
  [ 35  59  26]
  [ 30  11  23]
  [ 30  33  23]
  [ 40  31  25]
  [113  

In [18]:

# #####################
# METHOD #1: find the number of pixels that have (significantly) changed since the last frame
diff = cv.absdiff(img_hsv, prev_img)
_, diff = cv.threshold(diff, 10, 255, cv.THRESH_BINARY)
# cv.Threshold(diff, diff, 10, 255, cv.CV_THRESH_BINARY)
d_color = 0
for i in range(1, 3):
	d_color += float(cv.countNonZero(diff[:: ,:: , i])) / float(pixel_count)

if not BLACK_AND_WHITE:
	d_color = float(d_color/3.0) # 0..1

d_color


0.6490277777777779

In [26]:
# #####################
# METHOD #2: calculate the amount of change in the histograms
h_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
s_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
v_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
cv.split(img_hsv, [h_plane, s_plane, v_plane])
planes = [h_plane, s_plane, v_plane]

hist_size = [50, 50, 50]
hist_range = [0, 360, 0, 255, 0, 255]
# if not hist:
# 	hist = cvCreateHist(hist_size, cv.CV_HIST_ARRAY, hist_range, 1)
hist = cv.calcHist([img_hsv], [0, 1, 2], None, hist_size, hist_range)
# cv.NormalizeHist(hist, 1.0)
cv.normalize(hist, hist, 0, 1, cv.NORM_L2)

with np.printoptions(threshold=np.inf):
    print(hist)


[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [ ]:
def main():
	BLACK_AND_WHITE = False
	THRESHOLD = 0.48
	BW_THRESHOLD = 0.4
	
	os.chdir(sys.argv[1])
	try:
		os.mkdir(OUTPUT_DIR_NAME)
	except:
		pass
	
	if len(sys.argv) > 2:
		if sys.argv[2] == "bw":
			BLACK_AND_WHITE = True
			THRESHOLD = BW_THRESHOLD
			print("##########")
			print(" B/W MODE")
			print("##########")
	
	tree = et.parse("project.xml")
	movie = tree.getroot()
	file_path = movie.attrib["path"]
	cap = cv.VideoCapture(file_path)

	if DEBUG:
		cv.namedWindow("win", cv.WINDOW_AUTOSIZE)
		cv.moveWindow("win", 200, 200)

	hist = None
	prev_hist = None
	prev_img = None

	pixel_count = None
	frame_counter = 0

	last_frame_black = False
	black_frame_start = -1

	t = time.time()

	while 1:
		retval, img_orig = cap.read()
		
		if not retval: # eof
			cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (frame_counter-1), prev_img)
			"""movie.set("frames", str(frame_counter))
			tree.write("project.xml")"""
			break
		
		# img = cv.CreateImage((int(img_orig.width/4), int(img_orig.height/4)), cv.IPL_DEPTH_8U, 3)
		img = np.zeros((int(img_orig.shape[1]/4), int(img_orig.shape[0]/4), 3), np.uint8)

		img = cv.resize(img_orig, img.shape[:2], interpolation=cv.INTER_AREA)
		
		if frame_counter == 0: # erster frame
			cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (0), img)
			pixel_count = img.shape[0] * img.shape[1]
			prev_img = np.zeros(img.shape, np.uint8)
		
		if DEBUG and frame_counter % 2 == 1:
			cv.imshow("win", img)
		
		# img_hsv = np.zeros(img.shape, np.uint8)
		img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
		
		# #####################
		# METHOD #1: find the number of pixels that have (significantly) changed since the last frame
		diff = cv.absdiff(img_hsv, prev_img)
		_, diff = cv.threshold(diff, 10, 255, cv.THRESH_BINARY)
		# cv.Threshold(diff, diff, 10, 255, cv.CV_THRESH_BINARY)
		d_color = 0
		for i in range(1, 3):
			d_color += float(cv.countNonZero(diff[:: ,:: , i])) / float(pixel_count)
		
		if not BLACK_AND_WHITE:
			d_color = float(d_color/3.0) # 0..1

		# #####################
		# METHOD #2: calculate the amount of change in the histograms
		h_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
		s_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
		v_plane = cv.Mat(np.zeros(img.shape[:2], np.uint8))
		cv.split(img_hsv, [h_plane, s_plane, v_plane])
		planes = [h_plane, s_plane, v_plane]
		
		hist_size = [50, 50, 50]
		hist_range = [0, 360, 0, 255, 0, 255]
		# if not hist:
		# 	hist = cvCreateHist(hist_size, cv.CV_HIST_ARRAY, hist_range, 1)

		hist = cv.calcHist(img_hsv, [0, 1, 2], None, hist_size, hist_range)
		# cv.NormalizeHist(hist, 1.0)
		cv.normalize(hist, hist, 0, 1)

		print(hist[1])

		cv.waitKey()

		if prev_hist is None:
			# prev_hist = cv.CreateHist(hist_size, cv.CV_HIST_ARRAY, hist_range, 1)
			# wieso gibt es kein cv.CopyHist()?!
			prev_hist = cv.calcHist(planes, [1,1,1], None, hist_size, hist_range)
			# cv.NormalizeHist(prev_hist, 1.0)
			cv.normalize(prev_hist, prev_hist, 0, 1)
			continue
		
		d_hist = cv.compareHist(prev_hist, hist, cv.HISTCMP_INTERSECT)
		
		# combine both methods to make a decision
		if ((0.4*d_color + 0.6*(1-d_hist))) >= THRESHOLD:
			if DEBUG:
				if frame_counter % 2 == 0:
					cv.imshow("win", img)
				winsound.PlaySound(soundfile, winsound.SND_FILENAME|winsound.SND_ASYNC)
			print("%.3f" % ((0.4*d_color + 0.6*(1-d_hist))), "%.3f" % (d_color), "%.3f" % (1-d_hist), frame_counter)
			if DEBUG and DEBUG_INTERACTIVE:
				if win32api.MessageBox(0, "cut?", "", win32con.MB_YESNO) == 6: #yes
					cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (frame_counter), img)
			else:
				cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (frame_counter), img)
		
		prev_hist = cv.calcHist(planes, [1,1,1], None, hist_size, hist_range)
			
		# cv.NormalizeHist(prev_hist, 1.0)
		cv.normalize(prev_hist, prev_hist, 0, 1)
		
		# #####################
		# METHOD #3: detect series of (almost) black frames as an indicator for "fade to black"
		average = np.average(v_plane)
		if average <= 0.6:
			if not last_frame_black: # possible the start
				print("start", frame_counter)
				black_frame_start = frame_counter
			last_frame_black = True
		else:
			if last_frame_black: # end of a series of black frames
				cut_at = black_frame_start + int( (frame_counter - black_frame_start) / 2 )
				print("end", frame_counter, "cut at", cut_at)
				img_black = np.zeros((int(img_orig.shape[1]/4), int(img_orig.shape[0]/4), 3), np.uint8)
				img_black[::] = (0, 255, 0)
				cv.imwrite(OUTPUT_DIR_NAME + "\\%06d.png" % (cut_at), img_black)
			last_frame_black = False
		
		prev_img = img_hsv.copy()
		frame_counter += 1
		
		if DEBUG:
			if cv.waitKey(1) == 27:
				break
		

	if DEBUG:
		cv.destroyWindow("win");
	
	print("%.2f min" % ((time.time()-t) / 60))
	#print("- done -")
	return


# #########################
if __name__ == "__main__":
	main()
# #########################
